In [13]:
%load_ext dotenv
%dotenv
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

def fetch_metrics(url: str) -> str:
    return f"user1 response time: 300ms, user1 response time: 900ms: ({url})"

def fetch_logs(url: str) -> str:
    return f"user1 request size: 100 kb, user2 request size: 7887 kb: ({url})"

In [25]:
# ddg_search = DuckDuckGoSearchResults()

metrics_fetch_tool = Tool.from_function(
    func=fetch_metrics,
    name="MetricFetcher",
    description="Fetches response time metrics from client calls"
)

logs_fetch_tool = Tool.from_function(
    func=fetch_logs,
    name="LogsFetcher",
    description="Fetches logs with client request size in KB"
)

prompt_template = "Given response time by user and request size by user, suggest improvements for an API: {content}"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

suggester_tool = Tool.from_function(
    func=llm_chain.run,
    name="Suggester",
    description="Suggest API improvements based on metrics and logs"
)

tools = [logs_fetch_tool, metrics_fetch_tool, suggester_tool]

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)



In [26]:
prompt = "Evaluate the current metrics and propose an improvements. Use your tools to fetch metrics and logs about the response time of client calls."

print(agent.run(prompt))



> Entering new AgentExecutor chain...
I need to analyze the response time of client calls to identify any areas of improvement. I can use the MetricFetcher to fetch response time metrics and the LogsFetcher to fetch logs with client request size. Once I have the data, I can use the Suggester tool to suggest API improvements based on the metrics and logs.
Action: MetricFetcher
Action Input: Fetch response time metrics
Observation: user1 response time: 300ms, user1 response time: 900ms: (Fetch response time metrics)
Thought:I have fetched the response time metrics for the client calls. Now, I need to fetch the logs with client request size to get a complete understanding of the performance.
Action: LogsFetcher
Action Input: Fetch logs with client request size
Observation: user1 request size: 100 kb, user2 request size: 7887 kb: (Fetch logs with client request size)
Thought:I now have both the response time metrics and the client request size logs. I can use this data to identify any ar

In [28]:
prompt = "Which user is taking longer response time and why? Give some potential hypothesis"

print(agent.run(prompt))



> Entering new AgentExecutor chain...
To determine which user is experiencing longer response times and why, we need to fetch the response time metrics from client calls using the MetricFetcher tool.
Action: MetricFetcher
Action Input: Fetch response time metrics for all users
Observation: user1 response time: 300ms, user1 response time: 900ms: (Fetch response time metrics for all users)
Thought:Now that we have the response time metrics for all users, we can analyze the data to determine which user is experiencing longer response times and why. We can also generate potential hypotheses based on this information.
Action: Analyze response time metrics
Action Input: Response time metrics for all users
Observation: Analyze response time metrics is not a valid tool, try one of [LogsFetcher, MetricFetcher, Suggester].
Thought:Oops! It seems that "Analyze response time metrics" is not a valid tool. Let's try using the LogsFetcher to fetch logs with client request size in KB and analyze the

OutputParserException: Could not parse LLM output: `Now that we have generated potential API improvements based on the response time metrics and logs, we can analyze them to determine which user is experiencing longer response times and potential hypotheses. Based on the observations made earlier, user2 has a significantly longer response time compared to user1. One potential hypothesis could be that user2's larger request size (7887 KB) is causing the longer response time. However, further analysis is needed to confirm this hypothesis.`

In [17]:
# planner = load_chat_planner(llm)
# executor = load_agent_executor(llm, tools, verbose=True)
# agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [18]:
# result = agent.run("Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library.")
# print(result)